In [2]:
episodefile = open("../data/EpisodeList.txt","r")
episodeList = []

for line in episodefile:
    episodeList.append(line.strip().lower().replace(",",""))

SeasonList = [10,10,10,10,10,10,7,6]
episodeTuple = []
ekstranumb = 0
for numb in range(len(SeasonList)):
    for i in range(SeasonList[numb]):
        episodeTuple.append((numb+1, episodeList[i+ekstranumb]))
    ekstranumb += SeasonList[numb]

In [3]:
import lyricsgenius
genius = lyricsgenius.Genius("Apa5yYNZVRkIvn1izsQWN5l957pSR9jiG24s8ExAPKE9FHY9gdq4W8P7GPavWn1n")
artist = genius.search_artist("Game of Thrones")
artist.save_lyrics()

Searching for songs by Game of Thrones...

Song 1: "Winter is Coming"
Song 2: "Tyrion’s Courtroom Speech from The Laws of Gods and Men"
Song 3: "Kill the Boy"
Song 4: "Mother’s Mercy"
Song 5: "The Kingsroad"
Song 6: "The House of Black and White"
Song 7: "The Children"
Song 8: "Hardhome"
Song 9: "Liste des personnages"
Song 10: "The Wars to Come"
Song 11: "The Queen’s Justice"
Song 12: "Lord Snow"
Song 13: "High Sparrow"
Song 14: "The Dance of Dragons"
Song 15: "The Wolf and the Lion"
Song 16: "Cripples, Bastards and Broken Things"
Song 17: "The Winds of Winter"
Song 18: "Battle of the Bastards"
Song 19: "Book of the Stranger"
Song 20: "The Door"
Song 21: "The Red Woman"
Song 22: "You Win or You Die"
Song 23: "The North Remembers"
Song 24: "The Pointy End"
Song 25: "The Mountain and the Viper"
Song 26: "Unbowed, Unbent, Unbroken"
Song 27: "Oathbreaker"
Song 28: "The Gift"
Song 29: "Dragonstone"
Song 30: "A Golden Crown"
Song 31: "Fire and Blood"
Song 32: "The Rains of Castamere"
Song 3

In [3]:
import json
jFile = open("Lyrics_GameofThrones.json", "r")
j = json.load(jFile)

#Ignore this
#Dont even worry about it
#Just a simple Dict comprehension, no need to analyze it
LyricsListDict = {j["songs"][w]["title_with_featured"].lower().replace(",",""):j["songs"][w]["lyrics"].encode("UTF-8").decode("UTF-8") 
                    for w in range(len(j["songs"])) 
                    if j["songs"][w]["title_with_featured"].lower().replace(",","") in episodeList}



In [4]:
SeasonDict = dict()
#SeasonDict format
# {Title:{Season:SeasonNumber,Script:ScriptText}}
for season, title in episodeTuple:
    SeasonDict[title] = { "season": season,"script":LyricsListDict[title]}
print(SeasonDict['winter is coming'])

{'season': 1, 'script': '35 ContributorsWinter is Coming LyricsEPISODE 1 - WINTER IS COMING[First scene opens with three Rangers riding through a tunnel, leaving the Wall, and going into the woods. (Eerie music in background) One Ranger splits off and finds a campsite full of mutilated bodies, including a child hanging from a tree branch. A birds-eye view shows the bodies arranged in a shield-like pattern. The Ranger rides back to the other two.]\n\nWAYMAR ROYCE: What d’you expect? They’re savages. One lot steals a goat from another lot and before you know it, they’re ripping each other to pieces.\n\nWILL: I’ve never seen wildlings do a thing like this. I’ve never seen a thing like this, not ever in my life.\n\nWAYMAR ROYCE: How close did you get?\n\nWILL: Close as any man would.\n\nGARED: We should head back to the wall.\n\nROYCE: Do the dead frighten you?\n\nGARED: Our orders were to track the wildlings. We tracked them. They won’t trouble us no more.\n\nROYCE: You don’t think he’ll 

In [279]:

def sceneNameChecker(screenLine:str, i:int, lineSplit:str, sceneList:list):
    
    sceneName = None
    

    if lineSplit[0].upper() == "CUT TO":
        sceneName = lineSplit[1].strip()

    elif screenLine.startswith('EXT'):
        sceneName = screenLine.strip()[4:].strip()

    elif screenLine.startswith('INT'):
        sceneName = screenLine.strip()[4:].strip()

    elif screenLine.strip().startswith("--------"):
        sceneName = "Unnamed Scene"

    elif screenLine.upper().strip().startswith("CUT TO"):
        sceneName = screenLine.strip()[7:].strip()

    elif "SCENE SHIFTS" in screenLine.upper() or "SCENE CHANGES" in screenLine.upper():
        sceneName = "Unnamed Scene"
        
    elif "NEWSCENE" in screenLine.upper():
        sceneName = "Unnamed Scene"
    elif i == 0:
        
        sceneName = "Unnamed Scene"

    

    #Giving the sceneName an extra number if the sceneName occurs multiple times
    if sceneName is not None:
        if str(sceneName) in sceneList:
            try:
                sceneNameOld = sceneName
                sceneName = str(sceneName) + "_" + str(sceneList.count(sceneName))
                sceneList.append(str(sceneNameOld))
                
            except TypeError as error:
                print("Error:", sceneName)
            
        else:
            sceneList.append(str(sceneName))
        
        return True, str(sceneName), sceneList
       
        
    return False, False, False


def createDataframefromDict(screenplayDict:dict):
 
    dfcolumns = ['Person','SeasonNr','EpisodeID','EpisodeTitle','Scene','ScreenLine']


    sceneList = []
    allLines = []
    episodeID = 0
    

    for title in screenplayDict:

        episodeTitle = title
        episodeID += 1
        seasonNr = screenplayDict[title]['season']

        sceneName = None

        screenLines = screenplayDict[title]['script'].replace("\n\n\n", "NEWSCENE\n")
        screenLines = screenLines.split("\n")[1:]
        

        for i in range(len(screenLines)):
            
            line = screenLines[i]

            lineSplit = line.strip().split(":")
            
            if len(lineSplit) > 1:

                if sceneNameChecker(line,i,lineSplit,sceneList)[0] is True:
                    _, sceneName, sceneList = sceneNameChecker(line,i,lineSplit, sceneList)
                    #print("Scene:", sceneName, "episode:", episodeID)
                    if "Scene shifts" in str(line) and len(line[0].strip()) > 0:

                        print(str(line))                

                elif len(lineSplit) >= 2:
                    if sceneName is None:
                        _, sceneName, sceneList = sceneNameChecker(line,0,lineSplit, sceneList)

                    person = str(lineSplit[0].split("(")[0].strip()).upper()
                    screenLine = str(lineSplit[1])
                    
                    newEntry = [person, seasonNr, episodeID, episodeTitle, sceneName, screenLine]
                    allLines.append(newEntry)
                    
                
            else:
                
                if "SCENE SHIFTS" in str(line).upper() or "SCENE CHANGES" in str(line).upper() and len(line[0].strip()) > 0:
                    
                    _, sceneName, sceneList = sceneNameChecker(line,0,lineSplit, sceneList)
                    
            


    df = pd.DataFrame(allLines,columns=dfcolumns)
    return df

In [280]:
import pandas as pd
df = createDataframefromDict(SeasonDict)



df.dropna(*, axis=0)



In [341]:
lineDict = dict()

# LineDict -> key: Scene -> key: person -> value: lines
#                        -> value: seasonnr
#                        -> value: EpisodeID
#                        -> value: EpisodeTitle

for scene, person, lines, seasonNr, episodeID, episodeTitle in zip(df.Scene, df.Person, df.ScreenLine, df.SeasonNr, df.EpisodeID, df.EpisodeTitle):
    if scene not in lineDict:
        lineDict[scene] = {}
  
    if person not in lineDict[scene]:
        lineDict[scene][person] = {}
        lineDict[scene][person]["SeasonNr"] = seasonNr
        lineDict[scene][person]["EpisodeID"] = episodeID
        lineDict[scene][person]["EpisodeTitle"] = episodeTitle
        
    if "lines" not in lineDict[scene][person]: 
        lineDict[scene][person]["lines"] = []
    

    lineDict[scene][person]["lines"].append(lines)


print("Nr of scenes in dict:", len(lineDict))
#lineDict["Unnamed Scene"]




1039


In [372]:
import networkx as nx

G = nx.DiGraph()
PersonList = set(list(df.Person))
SceneList = set(list(df.Scene))

personList = list(lineDict[scene])
G.add_nodes_from(personList)

for scene in lineDict:    
    connectedPersons = list(lineDict[scene])

    for person in lineDict[scene]:
        lines = lineDict[scene][person]["lines"]
        
        if connectedPersons is not None:
            for name in connectedPersons:
                if person != name:
                    G.add_edge(person, name, lines = lines, weight = len(lines), seasonNr = lineDict[scene][person]["SeasonNr"], episodeTitle= lineDict[scene][person]["EpisodeTitle"])


print(len(G.nodes()))
print(G.edges())

728
[('MAN', 'CERSEI BARATHEON'), ('MAN', 'EDDARD STARK'), ('MAN', 'ROBERT BARATHEON'), ('MAN', 'IRRI'), ('MAN', 'DAENERYS TARGARYEN'), ('MAN', 'ROBB STARK'), ('MAN', 'BRAN STARK'), ('MAN', 'THEON GREYJOY'), ('MAN', 'OSHA'), ('MAN', 'WILDLING'), ('MAN', 'WILDLING 2'), ('MAN', 'TYRION LANNISTER'), ('MAN', 'MORD'), ('MAN', 'ARYA STARK'), ('MAN', 'SYRIO FOREL'), ('MAN', 'VISERYS TARGARYEN'), ('MAN', 'JORAH MORMONT'), ('MAN', 'JORA'), ('MAN', 'LYSA ARRYN'), ('MAN', 'ROBIN ARRYN'), ('MAN', 'CATELYN STARK'), ('MAN', 'KNIGHT 1'), ('MAN', 'KNIGHT 2'), ('MAN', 'KNIGHT 3'), ('MAN', 'KNIGHT 4'), ('MAN', 'KNIGHT 5'), ('MAN', 'SER VARDIS'), ('MAN', 'BRONN'), ('MAN', 'LANCEL LANNISTER'), ('MAN', 'RENLY BARATHEON'), ('MAN', 'BARRISTON'), ('MAN', 'BARRISTAN SELMY'), ('MAN', 'GRAND MAESTER PYCELLE'), ('MAN', 'PETYR BAELISH'), ('MAN', 'BERIC DONDARRION'), ('MAN', 'MORDANE'), ('MAN', 'SANSA STARK'), ('MAN', 'JOFFREY BARATHEON'), ('MAN', 'ROS'), ('MAN', 'KHAL DROGO'), ('MAN', 'BAELISH'), ('MAN', 'ROZ'), (

In [369]:
nx.draw_kamada_kawai(G)


Error in callback <function flush_figures at 0x7f1190c7bce0> (for post_execute):


KeyboardInterrupt: 